In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("insurance_pre.csv")

In [3]:
data

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
data=pd.get_dummies(data,drop_first=True)
data

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [5]:
data.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [6]:
indep=data[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=data[['charges']]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param={'criterion':['mse','mae'],'max_features': ['auto','sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestRegressor(),param,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    7.6s finished
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'],

In [10]:
y_pred=grid.predict(x_test)

In [11]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,y_pred)
r2

0.8721434383325972

In [12]:
result=grid.cv_results_
print("R2,Best result combination{}:".format(grid.best_params_))

R2,Best result combination{'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}:


In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.036689,0.002342,0.005006,1.123916e-06,mse,auto,10,"{'criterion': 'mse', 'max_features': 'auto', '...",0.778010,0.796660,0.790692,0.788454,0.007777,11
1,0.313504,0.020565,0.018010,2.163010e-03,mse,auto,100,"{'criterion': 'mse', 'max_features': 'auto', '...",0.807306,0.807407,0.795324,0.803346,0.005673,5
2,0.026683,0.002350,0.005002,1.236308e-06,mse,sqrt,10,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.801564,0.814866,0.772911,0.796447,0.017506,6
3,0.231805,0.002346,0.018346,2.363730e-03,mse,sqrt,100,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.812621,0.811755,0.798400,0.807592,0.006509,2
4,0.025016,0.004084,0.005002,6.836514e-07,mse,log2,10,"{'criterion': 'mse', 'max_features': 'log2', '...",0.787302,0.771457,0.792922,0.783894,0.009089,12
5,0.213437,0.004717,0.016672,2.358538e-03,mse,log2,100,"{'criterion': 'mse', 'max_features': 'log2', '...",0.808901,0.815658,0.802079,0.808879,0.005544,1
6,0.161743,0.002363,0.005002,6.836514e-07,mae,auto,10,"{'criterion': 'mae', 'max_features': 'auto', '...",0.797754,0.792604,0.782365,0.790908,0.006396,9
7,1.509730,0.008508,0.018339,2.358594e-03,mae,auto,100,"{'criterion': 'mae', 'max_features': 'auto', '...",0.797593,0.795702,0.795838,0.796378,0.000861,7
8,0.095038,0.004085,0.005003,5.947204e-07,mae,sqrt,10,"{'criterion': 'mae', 'max_features': 'sqrt', '...",0.802014,0.797383,0.767864,0.789087,0.015126,10
9,0.943390,0.002878,0.014516,7.052312e-04,mae,sqrt,100,"{'criterion': 'mae', 'max_features': 'sqrt', '...",0.815877,0.810089,0.791561,0.805842,0.010371,3


In [14]:
age=float(input("Age:"))
BMI=float(input("BMI:"))
children=int(input("Children:"))
sex=int(input("Male-1 or Female-0:"))
smoker=int(input("Yes-0 or No-1:"))


Age:45
BMI:25
Children:1
Male-1 or Female-0:1
Yes-0 or No-1:0


In [15]:
Future_charge_prediction=grid.predict([[age,BMI,children,sex,smoker]])
Future_charge_prediction
print("Future Prediction:{}".format(Future_charge_prediction))

Future Prediction:[15488.5280042]
